In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd /content/drive/MyDrive/pytorch

/content/drive/MyDrive/pytorch


In [98]:
import pandas as pd
import numpy as np
import tensorflow as tf

In [9]:
data = pd.read_csv("./sample_google_scholar.csv")

In [10]:
data = data.dropna()
data.head()

,author_name,email,affiliation,coauthors_names,research_interest
0,Lawrence Holder,wsu.edu,Washington State University,Diane J Cook##William Eberle,artificial_intelligence##machine_learning##dat...
3,Diane J Cook,eecs.wsu.edu,Washington State University,Lawrence Holder##Parisa Rashidi##Sajal K. Das#...,artificial_intelligence##machine_learning##sma...
4,Sumi Helal IEEE Fellow AAAS Fellow IET Fellow ...,cise.ufl.edu,University of Florida,Raja Bose##Darrell Woelk##Diane J Cook##Yousse...,digital_health##smart_homes##internet_of_thing...
5,Hani Hagras,essex.ac.uk,University of Essex,Christian Wagner,explainable_artificial_intelligence##ambient_i...
6,Anupam Joshi,umbc.edu,UMBC,Tim Finin##Yelena Yesha##Lalana Kagal##Dipanja...,data_management##mobile_computing##security##s...


In [11]:
# for features, it will convert first 10 characters of affliation into a vector of float
# by dividing each character by maximum axcii number

def convert_first_ten_characters_into_tensor(data):
  first_ten_characters = data[:10]
  converted = [ord(char)/256 for char in first_ten_characters]
  while len(converted) < 10:
    converted.append(0.0)
  return np.array(converted)

converted_affiliation = data['affiliation'].map(convert_first_ten_characters_into_tensor)
affiliation = np.vstack(converted_affiliation.values)
print(affiliation[:5])

[[0.33984375 0.37890625 0.44921875 0.40625    0.41015625 0.4296875
  0.40234375 0.453125   0.43359375 0.4296875 ]
 [0.33984375 0.37890625 0.44921875 0.40625    0.41015625 0.4296875
  0.40234375 0.453125   0.43359375 0.4296875 ]
 [0.33203125 0.4296875  0.41015625 0.4609375  0.39453125 0.4453125
  0.44921875 0.41015625 0.453125   0.47265625]
 [0.33203125 0.4296875  0.41015625 0.4609375  0.39453125 0.4453125
  0.44921875 0.41015625 0.453125   0.47265625]
 [0.33203125 0.30078125 0.2578125  0.26171875 0.         0.
  0.         0.         0.         0.        ]]


In [12]:
# for labels, it will be boolean valuel; True if email consist of '.edu' and False otherwise

converted_email = data['email'].str.contains('.edu')
labels = converted_email.values
labels[:5]

array([ True,  True,  True, False,  True])

In [88]:
full_df = pd.concat([pd.DataFrame(affiliation), pd.DataFrame(labels)], axis=1, ignore_index=True)
full_df.to_csv('data.csv', index=False, header=True)
full_df.head()

,0,1,2,3,4,5,6,7,8,9,10
0,0.339844,0.378906,0.449219,0.406250,0.410156,0.429688,0.402344,0.453125,0.433594,0.429688,True
1,0.339844,0.378906,0.449219,0.406250,0.410156,0.429688,0.402344,0.453125,0.433594,0.429688,True
2,0.332031,0.429688,0.410156,0.460938,0.394531,0.445312,0.449219,0.410156,0.453125,0.472656,True
3,0.332031,0.429688,0.410156,0.460938,0.394531,0.445312,0.449219,0.410156,0.453125,0.472656,False
4,0.332031,0.300781,0.257812,0.261719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,True


In [89]:
full_df['labels'] = full_df.iloc[:, -1].values
full_df['features'] = full_df.iloc[:, :-2].values.tolist()

for i in range(11):
  full_df.pop(i)

full_df.to_csv('data_2c.csv', index=False, header=True)

In [91]:
import tensorflow_datasets as tfds
import json

In [92]:
config = {}
config['path'] = "data.csv"
config['path_2c'] = "data_2c.csv"
config['tfds_dataset'] = "mnist"

In [99]:
class DataLoader:
  """ Data loader class """
  @staticmethod
  def load_data_tfds(config):
    """
    loads predefined dataset from tfds
    """
    print(config.get('tfds_dataset'))
    return tfds.load(config.get('tfds_dataset'), split=tfds.Split.TRAIN, as_supervised="True")

  @staticmethod
  def load_data_csv(config, tag, batch_size, label_name, select_columns):
    """
      load predefined dataset from tfds
    """
    return tf.data.experimental.make_csv_dataset(config.get(tag), batch_size=batch_size,
                                                 label_name=label_name, select_columns=select_columns)

  @staticmethod
  def load_data_from_nump_arrays(feature, label):
    """
    loads data from numpy to tf.data.Dataset
    """
    return tf.data.Dataset.from_tensor_slices(feature, label)

  @staticmethod
  def load_data_from_pandas(df):
    """
    loads data from pandas to tf.data.Dataset
    """
    return tf.data.Dataset.from_tensor_slices((df.iloc[:, :-1].values, df.iloc[:, -1].values))

In [95]:
label_name='labels'
select_columns = [0, 1]

In [100]:
data = DataLoader().load_data_csv(config, 'path_2c', 1, label_name, select_columns)

In [101]:
print(data)

<_PrefetchDataset element_spec=(OrderedDict([('features', TensorSpec(shape=(1,), dtype=tf.string, name=None))]), TensorSpec(shape=(1,), dtype=tf.string, name=None))>


In [102]:
for feature in data.take(1):
  print('Tensor:', feature)

Tensor: (OrderedDict([('features', <tf.Tensor: shape=(1,), dtype=string, numpy=
array([b'[0.32421875, 0.421875, 0.43359375, 0.4609375, 0.37890625, 0.41796875, 0.125, 0.33203125, 0.4296875, 0.41015625]'],
      dtype=object)>)]), <tf.Tensor: shape=(1,), dtype=string, numpy=array([b'False'], dtype=object)>)


In [103]:
label_name = '10'
select_columns = range(11)

In [105]:
data = DataLoader().load_data_csv(config, 'path', 1, label_name, select_columns)

In [106]:
print(data)

<_PrefetchDataset element_spec=(OrderedDict([('0', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('1', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('2', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('3', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('4', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('5', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('6', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('7', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('8', TensorSpec(shape=(1,), dtype=tf.float32, name=None)), ('9', TensorSpec(shape=(1,), dtype=tf.float32, name=None))]), TensorSpec(shape=(1,), dtype=tf.string, name=None))>


In [107]:
def feature_vector(samples, labels):
  return [x for x in list(samples.values())], labels

In [108]:
dataset = data.map(feature_vector)

In [109]:
print(dataset)

<_MapDataset element_spec=(TensorSpec(shape=(10, 1), dtype=tf.float32, name=None), TensorSpec(shape=(1,), dtype=tf.string, name=None))>


In [110]:
data = DataLoader().load_data_tfds(config)

mnist


Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


In [111]:
def data_generator(affiliation, labels):
  def fetch_examples():
    i=0
    while True:
      example = (affiliation[i], labels[i])
      i+=1
      i%=len(labels)
      yield example

  return fetch_examples

In [112]:
batch_size=5
features_shape=10

dataset_gen = tf.data.Dataset.from_generator(data_generator(affiliation, labels),
                                             output_types=(tf.float32, tf.bool),
                                             output_shapes=(tf.TensorShape(features_shape,),
                                                            tf.TensorShape(None))).batch(batch_size)

In [113]:
for feature in dataset_gen.take(1):
  print('Tensor:', feature)

Tensor: (<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[0.33984375, 0.37890625, 0.44921875, 0.40625   , 0.41015625,
        0.4296875 , 0.40234375, 0.453125  , 0.43359375, 0.4296875 ],
       [0.33984375, 0.37890625, 0.44921875, 0.40625   , 0.41015625,
        0.4296875 , 0.40234375, 0.453125  , 0.43359375, 0.4296875 ],
       [0.33203125, 0.4296875 , 0.41015625, 0.4609375 , 0.39453125,
        0.4453125 , 0.44921875, 0.41015625, 0.453125  , 0.47265625],
       [0.33203125, 0.4296875 , 0.41015625, 0.4609375 , 0.39453125,
        0.4453125 , 0.44921875, 0.41015625, 0.453125  , 0.47265625],
       [0.33203125, 0.30078125, 0.2578125 , 0.26171875, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ]],
      dtype=float32)>, <tf.Tensor: shape=(5,), dtype=bool, numpy=array([ True,  True,  True, False,  True])>)


In [114]:
from tensorflow import keras
from tensorflow.keras import layers

In [115]:
input_shape = 10
model = keras.Sequential(
    [
        keras.Input(shape=input_shape),
        layers.Dense(128, activation="relu", name="layer1"),
        layers.Dense(64, activation="relu", name="layer2"),
        layers.Dense(1, activation="sigmoid", name="layer3"),
    ]
)

In [116]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 128)               1408      
                                                                 
 layer2 (Dense)              (None, 64)                8256      
                                                                 
 layer3 (Dense)              (None, 1)                 65        
                                                                 
Total params: 9729 (38.00 KB)
Trainable params: 9729 (38.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [121]:
input_layer = layers.Input(10, name="input_layer")
x_1 = layers.Dense(128, activation="relu", name="layer1")(input_layer)
x_2 = layers.Dense(64, activation="relu", name="layer2")(x_1)
x_3 = layers.Dense(1, activation="sigmoid", name="layer3")(x_2)
model2 = keras.Model(input_layer, x_3)

In [122]:
model2.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_layer (InputLayer)    [(None, 10)]              0         
                                                                 
 layer1 (Dense)              (None, 128)               1408      
                                                                 
 layer2 (Dense)              (None, 64)                8256      
                                                                 
 layer3 (Dense)              (None, 1)                 65        
                                                                 
Total params: 9729 (38.00 KB)
Trainable params: 9729 (38.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [124]:
loss = tf.keras.losses.BinaryCrossentropy()

In [126]:
optimizer = tf.keras.optimizers.Adam()

In [127]:
model.compile(loss=loss, optimizer=optimizer)

In [128]:
model2.compile(loss=loss, optimizer=optimizer)

In [136]:
model.fit(affiliation, labels, batch_size=16, epochs=5, validation_split=0.2)

Epoch 1/5
3/3 [==============================] - 0s 59ms/step - loss: 0.6849 - val_loss: 0.7350
Epoch 2/5
3/3 [==============================] - 0s 32ms/step - loss: 0.6818 - val_loss: 0.7331
Epoch 3/5
3/3 [==============================] - 0s 29ms/step - loss: 0.6804 - val_loss: 0.7313
Epoch 4/5
3/3 [==============================] - 0s 25ms/step - loss: 0.6795 - val_loss: 0.7345
Epoch 5/5
3/3 [==============================] - 0s 22ms/step - loss: 0.6778 - val_loss: 0.7262


In [ ]:
model2.fit(affiliation, labels, batch_size=16, epochs=5, validation_split=0.2)

In [132]:
callback =tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', min_delta=0.1, patience=2, verbose=0,
    mode='min', baseline=None, restore_best_weights=False
)

In [137]:
optimizer=tf.keras.optimizers.Adam
loss_fn=tf.keras.losses.BinaryCrossentropy

BinaryAccuracy=tf.keras.metrics.BinaryAccuracy

epochs=5

for epoch in range(epochs):
  for step, (x_batch_train, y_batch_train) in enumerate(data):
    with tf.GradientTape() as tape:
      logits = model(x_batch_train, training=True)
      loss_ = loss_fn(y_batch_train, logits)
    grads = tape.gradient(loss_, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    BinaryAccuracy.update_state(y_batch_train, logits)
  print("epoch: " + str(epoch+1) + " loss" + str(loss_.numpy()))
  Acc = BinaryAccuracy.result()
  print("binary accuracy: ", float(Acc))
  BinaryAccuracy.reset_states()

ValueError: too many values to unpack (expected 2)